In [ ]:
%pip install torch transformers datasets pandas numpy scikit-learn openai gpt tensorflow

In [ ]:
import torch

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 64 # how many independent sequences will we process in parallel?
block_size = 256 # what is the maximum context length for predictions?
max_iters = 10000
eval_interval = 500
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 384
n_head = 6
n_layer = 6
dropout = 0.2
# ------------

torch.manual_seed(1618)

with open('/content/Julieth_model/Merged-QuestionsAnswers.csv', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]  # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l])  # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9 * len(data))  # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

# Save the model as an '.h5' file
def save_model(model, filename):
    torch.save(model.state_dict(), filename)
    print("Model saved successfully.")

# Test the model with a tech-related question
def test_model(model, question):
    model.eval()
    question_encoded = torch.tensor(encode(question), dtype=torch.long, device=device).unsqueeze(0)
    generated_answer = model.generate(question_encoded, max_new_tokens=100)[0].tolist()
    answer = decode(generated_answer)
    print("Question:", question)
    print("Answer:", answer)
    model.train()

@torch.no_grad()
def evaluate():
    model.eval()
    total_loss = 0.0
    total_accuracy = 0.0
    for k in range(eval_iters):
        X, Y = get_batch('val')
        logits, loss = model(X, Y)
        total_loss += loss.item()
        predicted = torch.argmax(logits, dim=-1)
        Y = Y.view(-1)  # Flatten the target tensor
        predicted = predicted.view(-1)  # Flatten the predicted tensor
        accuracy = torch.mean((predicted == Y).float())
        total_accuracy += accuracy.item()
    average_loss = total_loss / eval_iters
    average_accuracy = total_accuracy / eval_iters
    model.train()
    return average_loss, average_accuracy

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # input of size (batch, time-step, channels)
        # output of size (batch, time-step, head size)
        B,T,C = x.shape
        k = self.key(x)   # (B,T,hs)
        q = self.query(x) # (B,T,hs)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,hs)
        out = wei @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class GPTLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

        # better init, not covered in the original GPT video, but important, will cover in followup video
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
      # idx is (B, T) array of indices in the current context
      for _ in range(max_new_tokens):
          # crop idx to the last block_size tokens
          idx_cond = idx[:, -block_size:]
          # get the predictions
          logits, _ = self(idx_cond)
          # focus only on the last time step
          logits = logits[:, -1, :]  # becomes (B, C)
          # apply softmax to get probabilities
          probs = F.softmax(logits, dim=-1)  # (B, C)
          # sample from the distribution
          idx_next = torch.multinomial(probs, num_samples=1)  # (B, 1)
          # append sampled index to the running sequence
          idx = torch.cat((idx, idx_next), dim=1)  # (B, T+1)
      return idx


model = GPTLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # Every once in a while, evaluate the loss and accuracy on the train and validation sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        train_loss, train_accuracy = evaluate()
        val_loss, val_accuracy = evaluate()
        print(f"step {iter}: train loss {train_loss:.4f}, train accuracy {train_accuracy:.4f}, val loss {val_loss:.4f}, val accuracy {val_accuracy:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))
#open('more.txt', 'w').write(decode(m.generate(context, max_new_tokens=10000)[0].tolist()))


# Save the trained model
save_model(model, "model.h5")

# Test the model with a tech-related question
test_question = "What is it like to be an AI developer?"
test_model(model, test_question)

In [ ]:
import tensorflow as tf
from transformers import TFBertForMaskedLM, BertTokenizer

# Hyperparameters
batch_size = 64
block_size = 256
max_iters = 10000
eval_interval = 500
learning_rate = 3e-4
eval_iters = 200
n_embd = 384
n_head = 6
n_layer = 6
dropout = 0.2
device = 'cuda' if tf.config.experimental.list_physical_devices('GPU') else 'cpu'

tf.random.set_seed(1618)

with open('/content/Julieth_model/Merged-QuestionsAnswers.csv', 'r', encoding='utf-8') as f:
    text = f.read()

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
encoded_inputs = tokenizer.batch_encode_plus(
    [text],
    padding='max_length',
    truncation=True,
    max_length=block_size,
    return_tensors='tf'
)

input_ids = encoded_inputs['input_ids']
attention_mask = encoded_inputs['attention_mask']

# Train and test splits
data = input_ids[0]
n = int(0.9 * len(data))  # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = tf.random.uniform((batch_size,), maxval=len(data) - block_size, dtype=tf.int32)
    x = tf.stack([data[i:i+block_size] for i in ix])
    y = tf.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

# Save the model as an '.h5' file
def save_model(model, filename):
    model.save_weights(filename)
    print("Model saved successfully.")

# Test the model with a tech-related question
def test_model(model, question):
    question_encoded = tokenizer.encode_plus(
        question,
        padding='max_length',
        truncation=True,
        max_length=block_size,
        return_tensors='tf'
    )
    generated_answer = model.generate(question_encoded.input_ids, max_new_tokens=100)[0].numpy()
    answer = tokenizer.decode(generated_answer)
    print("Question:", question)
    print("Answer:", answer)

def evaluate():
    total_loss = 0.0
    total_accuracy = 0.0
    for k in range(eval_iters):
        X, Y = get_batch('val')
        logits = model(X)
        loss = tf.reduce_mean(tf.keras.losses.sparse_categorical_crossentropy(Y, logits, from_logits=True))
        total_loss += loss.numpy()
        predicted = tf.argmax(logits, axis=-1)
        accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.float32))
        total_accuracy += accuracy.numpy()
    average_loss = total_loss / eval_iters
    average_accuracy = total_accuracy / eval_iters
    return average_loss, average_accuracy

class BertLanguageModel(tf.keras.Model):

    def __init__(self):
        super(BertLanguageModel, self).__init__()
        self.bert = TFBertForMaskedLM.from_pretrained('bert-base-uncased')

    def call(self, inputs):
        outputs = self.bert(inputs)
        logits = outputs.logits
        return logits

    def generate(self, inputs, max_new_tokens):
        context = inputs
        for _ in range(max_new_tokens):
            logits = self(context)
            logits = logits[:, -1, :]  # shape: (batch_size, vocab_size)
            probs = tf.nn.softmax(logits, axis=-1)  # apply softmax to get probabilities
            idx_next = tf.random.categorical(probs, num_samples=1, dtype=tf.int32)  # sample from the distribution
            context = tf.concat([context, idx_next], axis=1)  # append sampled index to the running sequence
        return context

model = BertLanguageModel()

# Create a TensorFlow optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

for iter in range(max_iters):
    # Every once in a while, evaluate the loss and accuracy on the train and validation sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        train_loss, train_accuracy = evaluate()
        val_loss, val_accuracy = evaluate()
        print(f"step {iter}: train loss {train_loss:.4f}, train accuracy {train_accuracy:.4f}, val loss {val_loss:.4f}, val accuracy {val_accuracy:.4f}")

    # Sample a batch of data
    xb, yb = get_batch('train')

    # Evaluate the loss
    with tf.GradientTape() as tape:
        logits = model(xb)
        loss = tf.reduce_mean(tf.keras.losses.sparse_categorical_crossentropy(yb, logits, from_logits=True))

    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

# Generate from the model
context = tf.zeros((1, block_size), dtype=tf.int32)
generated_sequence = model.generate(context, max_new_tokens=500)[0].numpy()
decoded_sequence = tokenizer.decode(generated_sequence)
print(decoded_sequence)

# Save the trained model
save_model(model, "model_x.h5")

# Test the model with a tech-related question
test_question = "What is it like to be an AI developer?"
test_model(model, test_question)
